In [2]:
from IPython.core.display import display, HTML

html_code = """
<!DOCTYPE html>
<html>
<head>
    <h2>Mental Health Risk Assessment UI</h2>
</head>
<body>
    <form>
        <label>Growing Stress:</label><br>
        <input type="number" name="stress" min="0" max="10"><br><br>

        <label>Mood Swings:</label><br>
        <input type="number" name="mood" min="0" max="10"><br><br>

        <label>Social Weakness:</label><br>
        <input type="number" name="social" min="0" max="10"><br><br>

        <label>Model Selection:</label><br>
        <select name="model">
            <option value="logistic">Logistic Regression</option>
            <option value="tabular_nn">Tabular Neural Network</option>
            <option value="ensemble">Fusion/Ensemble</option>
        </select><br><br>

        <input type="submit" value="Predict Risk">
    </form>

    <h3>Prediction Result:</h3>
    <p>[Model Output Here]</p>

    <h3>Feature Importance:</h3>
    <p>[Top Contributing Factors Displayed Here]</p>

    <button>Export Report</button>
</body>
</html>
"""

# Display the UI
display(HTML(html_code))

### Mental Health Risk Assessment UI with Model Integration

In [25]:
# Step 1: Install and Import Libraries

# Install required libraries
#!pip install torch torchvision torchaudio
#!pip install scikit-learn

import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from google.colab import output
from ipywidgets import widgets
import torch
import torch.nn as nn
import torch.optim as optim
from IPython.core.display import display, HTML

In [19]:
# Step 2: Load Real Data & Train Models of Logistic Regression and Tabular Neural Network


X_train_lr = pd.read_csv('/content/X_train_encoded_scaled.csv')
y_train_lr = pd.read_csv('/content/y_train.csv').squeeze()

X_test_lr = pd.read_csv('/content/X_test_encoded_scaled.csv')
y_test_lr = pd.read_csv('/content/y_test.csv').squeeze()

# Step 2: Load Data for Tabular Neural Network (TT)
X_train_tt = pd.read_csv('/content/test_X_train_encoded_scaled.csv')  # Same training data
y_train_tt = pd.read_csv('/content/test_y_train.csv').squeeze()

X_test_tt = pd.read_csv('/content/test_X_test_encoded_scaled.csv')
y_test_tt = pd.read_csv('/content/test_y_test.csv').squeeze()

# ===========================
# Train the Models
# ===========================

# Logistic Regression Model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_lr, y_train_lr)

# Tabular Neural Network (MLP as proxy)
tt_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=50, random_state=42)
tt_model.fit(X_train_tt, y_train_tt)

print("Logistic Regression Data Shapes:")
print("X_train_lr:", X_train_lr.shape)
print("y_train_lr:", y_train_lr.shape)
print("X_test_lr:", X_test_lr.shape)
print("y_test_lr:", y_test_lr.shape)

print("\nTabular Neural Network Data Shapes:")
print("X_train_tt:", X_train_tt.shape)
print("y_train_tt:", y_train_tt.shape)
print("X_test_tt:", X_test_tt.shape)
print("y_test_tt:", y_test_tt.shape)

Logistic Regression Data Shapes:
X_train_lr: (182690, 16)
y_train_lr: (182690,)
X_test_lr: (78296, 16)
y_test_lr: (78296,)

Tabular Neural Network Data Shapes:
X_train_tt: (182690, 11)
y_train_tt: (182690,)
X_test_tt: (78296, 11)
y_test_tt: (78296,)


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
# ===========================
# Step 3: Define the UI Widgets
# ===========================
from IPython.core.display import display, HTML

title = widgets.HTML(value="<h3>Mental Health Risk Assessment UI</h3>")

# Input fields
stress_input = widgets.FloatText(description='Growing Stress:', value=0.0)
mood_input = widgets.FloatText(description='Mood Swings:', value=0.0)
social_input = widgets.FloatText(description='Social Weakness:', value=0.0)

# Model Selection
model_dropdown = widgets.Dropdown(
    options=['Logistic Regression', 'Tabular Neural Network'],
    value='Logistic Regression',
    description='Model Selection:'
)

# Prediction button and result label
predict_button = widgets.Button(description="Predict Risk", button_style='info')
result_label = widgets.HTML(value="<b>Prediction Result:</b><br>[Model Output Here]")


In [35]:
# ===========================
# Step 4: Define the Prediction Logic (Final Version)
# ===========================
from IPython.display import clear_output, display
from ipywidgets import VBox

def predict_risk():
    # Clear previous output to avoid duplicates
    clear_output(wait=True)

    # Display the UI again after clearing output
    display(VBox([title,
                  stress_input,
                  mood_input,
                  social_input,
                  model_dropdown,
                  predict_button,
                  result_label]))

    # Get user inputs
    stress = stress_input.value
    mood = mood_input.value
    social = social_input.value
    selected_model = model_dropdown.value

    if selected_model == 'Logistic Regression':
        # Create input vector matching the logistic regression feature space
        input_vector = np.zeros(X_train_lr.shape[1])
        lr_features = X_train_lr.columns.tolist()

        # Map selected fields to correct positions
        for feature, value in zip(['Growing_Stress', 'Mood_Swings', 'Social_Weakness'], [stress, mood, social]):
            if feature in lr_features:
                input_vector[lr_features.index(feature)] = value

        # Predict
        input_df_lr = pd.DataFrame([input_vector], columns=X_train_lr.columns)
        pred = lr_model.predict(input_df_lr)[0]
        pred_proba = lr_model.predict_proba(input_df_lr).max()

        result_label.value = (f"<b>Prediction Result:</b><br>"
                              f"Logistic Regression predicts risk class: <b>{pred}</b><br>"
                              f"<b>Confidence:</b> {pred_proba:.2f}")

    elif selected_model == 'Tabular Neural Network':
        # Create input vector matching the tabular NN feature space
        input_vector = np.zeros(X_train_tt.shape[1])
        tt_features = X_train_tt.columns.tolist()

        for feature, value in zip(['Growing_Stress', 'Mood_Swings', 'Social_Weakness'], [stress, mood, social]):
            if feature in tt_features:
                input_vector[tt_features.index(feature)] = value

        # Predict
        input_df_tt = pd.DataFrame([input_vector], columns=X_train_tt.columns)
        pred = tt_model.predict(input_df_tt)[0]
        pred_proba = tt_model.predict_proba(input_df_tt).max()
        result_label.value = (f"<b>Prediction Result:</b><br>"
                              f"Tabular Neural Network predicts risk class: <b>{pred}</b><br>"
                              f"<b>Confidence:</b> {pred_proba:.2f}")

    else:
        result_label.value = "<b>Error:</b> Please select a valid model."

In [36]:
# ===========================
# Step 5: Connect UI to Logic
# ===========================
from ipywidgets import VBox # Import VBox

predict_button.on_click(lambda b: predict_risk())

# Display the UI
display(VBox([title,
              stress_input,
              mood_input,
              social_input,
              model_dropdown,
              predict_button,
              result_label]))

In [56]:
# Gradio Integration (Logistic + Tabular NN)

import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import joblib

# ======================
# Load Models & Data
# ======================
# Logistic Regression data
X_train_lr = pd.read_csv('/content/X_train_encoded_scaled.csv')

# Tabular Neural Network data
X_train_tt = pd.read_csv('/content/test_X_test_encoded_scaled.csv')

# ======================
# Visualization Function
# ======================
def plot_probabilities(probas, classes):
    fig, ax = plt.subplots(figsize=(5,3))
    colors = ["green", "orange", "red"]
    ax.bar(classes, probas, color=colors)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Probability")
    ax.set_title("Risk Class Probabilities")

    for i, v in enumerate(probas):
        ax.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=12)

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format="png")
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode()
    plt.close(fig)

    return f"<img src='data:image/png;base64,{img_base64}'/>"


# ======================
# Prediction Logic
# ======================

def predict_risk_web_visual(stress, mood, social, model_choice):
    if model_choice == "Logistic Regression":
        input_vector = np.zeros(X_train_lr.shape[1])
        lr_features = X_train_lr.columns.tolist()

        # Map input features
        if 'Growing_Stress' in lr_features:
            input_vector[lr_features.index('Growing_Stress')] = stress
        if 'Mood_Swings' in lr_features:
            input_vector[lr_features.index('Mood_Swings')] = mood
        if 'Social_Weakness' in lr_features:
            input_vector[lr_features.index('Social_Weakness')] = social

        input_df = pd.DataFrame([input_vector], columns=lr_features)
        pred = lr_model.predict(input_df)[0]
        probas = lr_model.predict_proba(input_df)[0]

    else:  # Tabular Neural Network
        input_vector = np.zeros(X_train_tt.shape[1])
        tt_features = X_train_tt.columns.tolist()

        # Map input features
        if 'Growing_Stress' in tt_features:
            input_vector[tt_features.index('Growing_Stress')] = stress
        if 'Mood_Swings' in tt_features:
            input_vector[tt_features.index('Mood_Swings')] = mood
        if 'Social_Weakness' in tt_features:
            input_vector[tt_features.index('Social_Weakness')] = social

        input_df = pd.DataFrame([input_vector], columns=tt_features)
        pred = tt_model.predict(input_df)[0]
        probas = tt_model.predict_proba(input_df)[0]

   # =====================
    # Output Formatting
    # =====================
    risk_labels = {0: ("Low Risk", "green"), 1: ("Moderate Risk", "orange"), 2: ("High Risk", "red")}
    label, color = risk_labels[pred]

    result_html = f"""
    <div style='font-size:20px; font-weight:bold; color:{color};'>
        Predicted Class: {pred} ({label})<br>
        Confidence: {max(probas):.2f}
    </div>
    """

    # Plot
    class_labels = ["Low", "Moderate", "High"]
    prob_chart = plot_probabilities(probas, class_labels)

    return result_html + prob_chart

In [57]:
# Gradio UI Change

with gr.Blocks() as demo:
    gr.Markdown("## Mental Health Risk Assessment Web App")

    with gr.Row():
        with gr.Column():
            stress = gr.Number(label="Growing Stress", value=0)
            mood = gr.Number(label="Mood Swings", value=0)
            social = gr.Number(label="Social Weakness", value=0)
            model_choice = gr.Dropdown(["Logistic Regression", "Tabular Neural Network"], label="Model Selection", value="Logistic Regression")

            submit = gr.Button("Submit", elem_id="submit-button", elem_classes="btn-primary")
            clear = gr.Button("Clear")

        with gr.Column():
            output = gr.HTML()

    # Actions
    submit.click(predict_risk_web_visual,
                 inputs=[stress, mood, social, model_choice],
                 outputs=output)

    clear.click(lambda: (0, 0, 0, "Logistic Regression", ""),
                inputs=[],
                outputs=[stress, mood, social, model_choice, output])

# ======================
# Launch App
# ======================
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://82a375ba022812bcec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> None
Killing tunnel 127.0.0.1:7865 <> https://30ebd7fa3ecfd91403.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://0adfc76994adbf2e94.gradio.live
Killing tunnel 127.0.0.1:7867 <> https://bea52852eab751c626.gradio.live
Killing tunnel 127.0.0.1:7868 <> https://82a375ba022812bcec.gradio.live


In [61]:
#  Enhanced Gradio App Code

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import gradio as gr
import random

# Simulated models (replace with real models in production)
feature_names = ["Growing Stress", "Mood Swings", "Social Weakness"]
lr_coef = np.array([0.4, 0.3, 0.3])
tt_coef = np.array([0.35, 0.25, 0.4])

# Simulate dynamic probabilities
def simulate_proba(inputs):
    total = sum(inputs) + 0.01
    prob_low = max(0.0, 1 - total / 12)
    prob_high = min(1.0, total / 12)
    prob_med = 1 - prob_low - prob_high
    return [round(prob_low, 2), round(prob_med, 2), round(prob_high, 2)]

# Plot bar chart as image string
def plot_contributions(contribs, feature_names):
    fig, ax = plt.subplots()
    colors = []
    for c in contribs:
        if c >= 0.5:
            colors.append("red")
        elif c >= 0.25:
            colors.append("orange")
        else:
            colors.append("green")
    bars = ax.bar(feature_names, contribs, color=colors)
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.2f}', ha='center', fontsize=10)
    ax.set_ylabel("Contribution Score")
    ax.set_title("Risk Contribution by Factor")
    plt.ylim(0, 1)

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode()
    plt.close(fig)
    return f"<img src='data:image/png;base64,{img_base64}'/>"

# Main prediction function
def predict_risk_final(stress, mood, social, model_choice):
    inputs = np.array([stress, mood, social])

    if model_choice == "Logistic Regression":
        contrib_raw = inputs * lr_coef
    else:
        contrib_raw = inputs * tt_coef

    contribs = contrib_raw / (np.sum(contrib_raw) + 0.001)  # avoid zero division

    # Simulate dynamic probabilities
    probas = simulate_proba(inputs)
    pred = np.argmax(probas)

    # Risk labels and colors
    risk_labels = {0: ("Low Risk", "green"), 1: ("Moderate Risk", "orange"), 2: ("High Risk", "red")}
    label, color = risk_labels[pred]

    # Recommendations
    recs = {
        0: "Maintain healthy habits and monitor stress levels.",
        1: "Consider talking to a counselor. Monitor your mood and social interactions.",
        2: "Immediate consultation recommended. Prioritize stress management and seek support."
    }

    # Determine factor colors dynamically
    factor_colors = []
    for c in contribs:
        if c >= 0.5:
            factor_colors.append("red")
        elif c >= 0.25:
            factor_colors.append("orange")
        else:
            factor_colors.append("green")

    # Create output HTML
    result_html = f"""
    <div style='font-size:20px; font-weight:bold; color:{color};'>
        Predicted Class: {pred} ({label})<br>
        Confidence: {max(probas):.2f}<br><br>
    </div>
    <div style='font-size:16px;'>
        <b>Main Risk Factors:</b><br>
    """
    for i, f in enumerate(feature_names):
        result_html += f"<span style='color:{factor_colors[i]};'>- {f}: {contribs[i]*100:.1f}% contribution</span><br>"

    result_html += f"<br><b>Recommendation:</b> {recs[pred]}</div>"

    # Chart
    contrib_chart = plot_contributions(contribs, feature_names)

    return result_html + contrib_chart

In [64]:
# Gradio Interface
iface = gr.Interface(
    fn=predict_risk_final,
    inputs=[
        gr.Number(label="Growing Stress", value=1),
        gr.Number(label="Mood Swings", value=1),
        gr.Number(label="Social Weakness", value=1),
        gr.Dropdown(["Logistic Regression", "Tabular Neural Network"], label="Model Selection")
    ],
    outputs=gr.HTML(),
    allow_flagging="never",
    title="Mental Health Risk Assessment Web App"
)

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fe27b9453114414b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
